<a href="https://colab.research.google.com/github/hvmarco/PV-feasibility/blob/main/project_marcohernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PV feasibility analysis
This is the project for the course <i>Python Foundations for Spatial Analysis (2024)</i> taught by Ujaval Gandhi and: Vigna Purohi at Spatial Thoughts.

## Setup and data download

First we need to install some additional libraries and import them.

In [1]:
#install and import necessary packages
%%capture
# use %%capture to prevent printing the output of the cell
!pip install numpy-financial #to use financial functions previously in numpy
!pip install pvlib[optional]  #toolbox for simulating the performance of photovoltaic energy systems https://pvlib-python.readthedocs.io/en/stable/index.html#

import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import pvlib


Since we will be using some files with the data (location, components, etc), we clone the repository in the colab environment to import them.

In [2]:
%%capture
#clone the repository to include the files needed
!git clone https://github.com/hvmarco/PV-feasibility.git


In [3]:
#change the path in colab so we can access the files
DATA_FOLDER = 'PV-feasibility'
%cd $DATA_FOLDER

/content/PV-feasibility


## Define location

In [10]:
import time
for x in range(3):
    print(x)
    time.sleep(2) #like this we can ensure that we only send 30 requests per minute

0
1
2


In [13]:
# PVGIS has a limit of 30 calls/second
url_base = f"https://re.jrc.ec.europa.eu/api/MRcalc?"

lat=45
lon=8

#----- set API call parameters ------------------
pvgis_params = dict(
    lat=lat,
    lon=lon,
    usehorizon=1,
    raddatabase='PVGIS-ERA5',
    horirrad=1,
    avtemp=1,
    outputformat='json')

params = "&".join([f'{key}={value}' for key, value in pvgis_params.items()])
url_MRcalc = f'{url_base}&{params}'

pvgis_params

{'lat': 45,
 'lon': 8,
 'usehorizon': 1,
 'raddatabase': 'PVGIS-ERA5',
 'horirrad': 1,
 'avtemp': 1,
 'outputformat': 'json'}

In [12]:
parameters = {
        'lat': 'source_coordinates[0]',
        'lon' : 'source_coordinates[1]',
        'usehorizon' : 1,
        'raddatabase' : 'PVGIS-ERA5',
        'horirrad' : 1,
        'avtemp' : 1,
        'outputformat' :'json'
    }

params = "&".join([f'{key}={value}' for key, value in parameters.items()])
url_MRcalc = f'{url_base}&{params}'
params

'lat=source_coordinates[0]&lon=source_coordinates[1]&usehorizon=1&raddatabase=PVGIS-ERA5&horirrad=1&avtemp=1&outputformat=json'

In [21]:
stockholm = (59.3293, 18.0686)

url_base = f"https://re.jrc.ec.europa.eu/api/MRcalc?" #specific for monthly radiation
    # list of parameters to send to the API
pvgis_params = {
            'lat': stockholm[0],
            'lon' : stockholm[1],
            'usehorizon' : 1,
            'raddatabase' : 'PVGIS-ERA5',
            'horirrad' : 1,
            'avtemp' : 1,
            'outputformat' :'json'
            }
params = "&".join([f'{key}={value}' for key, value in pvgis_params.items()]) #concatenate the elements
url_MRcalc = f'{url_base}&{params}'
url_MRcalc

#response = requests.get(url_MRcalc)
#response

'https://re.jrc.ec.europa.eu/api/MRcalc?&lat=59.3293&lon=18.0686&usehorizon=1&raddatabase=PVGIS-ERA5&horirrad=1&avtemp=1&outputformat=json'

In [22]:
import json  # To convert a JSON strings from APIS to Python objects
import requests # The requests module allows you to send HTTP requests and parse the responses using Python.
import time

stockholm = (59.3293, 18.0686)

def get_pvgis_tmy(source_coordinates):
    url_base = f"https://re.jrc.ec.europa.eu/api/MRcalc?" #specific for monthly radiation
    # list of parameters to send to the API
    pvgis_params = {
            'lat': source_coordinates[0],
            'lon' : source_coordinates[1],
            'usehorizon' : 1,
            'raddatabase' : 'PVGIS-ERA5',
            'horirrad' : 1,
            'avtemp' : 1,
            'outputformat' :'json'
            }
    params = "&".join([f'{key}={value}' for key, value in pvgis_params.items()]) #concatenate the elements
    url_MRcalc = f'{url_base}&{params}'

    response = requests.get(url_MRcalc)

    if response.status_code == 200:
        data = response.json()
        #row_json = json.loads(response.text)
        return data
    else:
        print('Request failed.')
        return -9999

get_month_rad(stockholm)


{'inputs': {'location': {'latitude': 59.3293,
   'longitude': 18.0686,
   'elevation': 18.0},
  'meteo_data': {'radiation_db': 'PVGIS-ERA5',
   'meteo_db': 'ERA-Interim',
   'year_min': 2005,
   'year_max': 2016,
   'use_horizon': True,
   'horizon_db': None,
   'horizon_data': 'DEM-calculated'},
  'plane': {'fixed_horizontal': {'slope': {'value': 0, 'optimal': '-'},
    'azimuth': {'value': '-', 'optimal': '-'}}}},
 'outputs': {'monthly': [{'year': 2005,
    'month': 1,
    'H(h)_m': 11.46,
    'T2m': 1.4},
   {'year': 2005, 'month': 2, 'H(h)_m': 26.8, 'T2m': -2.1},
   {'year': 2005, 'month': 3, 'H(h)_m': 83.09, 'T2m': -2.0},
   {'year': 2005, 'month': 4, 'H(h)_m': 136.91, 'T2m': 5.6},
   {'year': 2005, 'month': 5, 'H(h)_m': 161.66, 'T2m': 10.4},
   {'year': 2005, 'month': 6, 'H(h)_m': 165.35, 'T2m': 14.8},
   {'year': 2005, 'month': 7, 'H(h)_m': 173.29, 'T2m': 18.6},
   {'year': 2005, 'month': 8, 'H(h)_m': 129.96, 'T2m': 16.3},
   {'year': 2005, 'month': 9, 'H(h)_m': 88.06, 'T2m': 13

## PV components

In [14]:
from pvlib import pvsystem
#import the components databases as pandas Dataframes formated so that pvlib can use
moddb = pvsystem.retrieve_sam(path = 'CEC Modules.csv')
invdb = pvsystem.retrieve_sam(path = 'CEC Inverters.csv')

#select the module and inverter we will be using
pv_module = moddb['Trina_Solar_TSM_670DEG21C_20']
inverter = invdb['Sungrow_Power_Supply_Co___Ltd___SG250HX_US__800V_']


/usr/local/lib/python3.10/dist-packages/pvlib/pvsystem.py:2085: UserWarning: Original names contain 29 duplicate(s).
  warnings.warn('Original names contain %d duplicate(s).' % n_duplicates)
/usr/local/lib/python3.10/dist-packages/pvlib/pvsystem.py:2089: UserWarning: Normalized names contain 29 duplicate(s).
  warnings.warn(
